# Projet 4 : **Anticiper le retard de vols des avions**

## **1.Libraries**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.lines import Line2D
import seaborn as sns
from scipy import stats
from sklearn import preprocessing
from os import listdir
import glob
import missingno as msno
from feature_engine.categorical_encoders import OneHotCategoricalEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
import itertools
import pickle 
from sklearn.externals import joblib
import gzip

## **2.Data Set**

In [2]:
df_model_sc = pd.read_csv('05_data_model_final_sc.csv')
df_model_sc = df_model_sc.drop(['Unnamed: 0'], axis=1)
df_model_sc.head()

,DEP_DELAY,CARRIER_AA,CARRIER_UA,CARRIER_WN,CARRIER_EV,CARRIER_B6,CARRIER_NK,CARRIER_DL,CARRIER_AS,CARRIER_OO,...,CRS_DEP_TIME_8,CRS_DEP_TIME_23,CRS_DEP_TIME_0,CRS_DEP_TIME_3,CRS_DEP_TIME_2,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,-3,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,-0.720957,-0.065652,1.835812,-0.026497,-0.773717
1,39,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,2.741342,-0.065652,2.683872,-0.026497,-0.773717
2,20,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1.054581,-0.065652,-0.623562,-0.026497,-0.773717
3,17,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,-0.720957,-0.065652,0.394110,-0.026497,0.056031
4,20,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0.877027,-0.065652,-0.793174,-0.026497,-0.773717


In [1]:
df_delay = pd.read_csv('06_data_delay.csv')
df_delay = df_delay.drop(['Unnamed: 0'], axis=1)
df_delay.head()

## **3. Final Model**

### 3.1 Train KNN support to predict delay variables

In [4]:
from sklearn.neighbors import KNeighborsRegressor
features = df_model_sc.drop(['DEP_DELAY', 'CARRIER_DELAY','WEATHER_DELAY','NAS_DELAY','SECURITY_DELAY','LATE_AIRCRAFT_DELAY'], axis=1).values

reg_knn = KNeighborsRegressor()
params = {'n_neighbors':[7]}

target_knn_carrier_delay = df_delay['CARRIER_DELAY'].values
X_train_knn_carrier_delay, X_test_knn_carrier_delay, y_train_knn_carrier_delay, y_test_knn_carrier_delay = train_test_split(features, target_knn_carrier_delay, test_size = 0.25)
gscv_reg_carrier_delay_knn= GridSearchCV(reg_knn, params)
gscv_reg_carrier_delay_knn.fit(X_train_knn_carrier_delay, y_train_knn_carrier_delay)


target_knn_NAS_delay = df_delay['NAS_DELAY'].values
X_train_knn_NAS_delay, X_test_knn_NAS_delay, y_train_knn_NAS_delay, y_test_knn_NAS_delay = train_test_split(features, target_knn_NAS_delay, test_size = 0.25)
gscv_reg_NAS_delay_knn= GridSearchCV(reg_knn, params)
gscv_reg_NAS_delay_knn.fit(X_train_knn_NAS_delay, y_train_knn_NAS_delay)


target_knn_AIRCRAFT_delay = df_delay['LATE_AIRCRAFT_DELAY'].values
X_train_knn_AIRCRAFT_delay, X_test_knn_AIRCRAFT_delay, y_train_knn_AIRCRAFT_delay, y_test_knn_AIRCRAFT_delay = train_test_split(features, target_knn_AIRCRAFT_delay, test_size = 0.25)
gscv_reg_AIRCRAFT_delay_knn= GridSearchCV(reg_knn, params)
gscv_reg_AIRCRAFT_delay_knn.fit(X_train_knn_AIRCRAFT_delay, y_train_knn_AIRCRAFT_delay)

GridSearchCV(cv=None, error_score='raise',
       estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_neighbors': [7]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True, scoring=None, verbose=0)

### 3.2 Train Ridge to predict depart delay 

In [5]:
X_model_rd = df_model_sc.drop(['DEP_DELAY','SECURITY_DELAY', 'WEATHER_DELAY'], axis=1).values
y_model_rd = df_model_sc['DEP_DELAY'].values

In [6]:
X_train_rd, X_test_rd, y_train_rd, y_test_rd = train_test_split(X_model_rd, y_model_rd, test_size = 0.25)
X_train_rd.shape, X_test_rd.shape

((10233, 630), (3412, 630))

In [7]:
from sklearn.linear_model import Ridge
reg_ridge =  Ridge()
params = {'alpha':np.logspace(-5,5,20)}
#params = {'alpha':69.51927961775606}
gscv_reg_ridge=GridSearchCV(reg_ridge,params, cv=5)
gscv_reg_ridge.fit(X_train_rd, y_train_rd)

GridSearchCV(cv=5, error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'alpha': array([1.00000e-05, 3.35982e-05, 1.12884e-04, 3.79269e-04, 1.27427e-03,
       4.28133e-03, 1.43845e-02, 4.83293e-02, 1.62378e-01, 5.45559e-01,
       1.83298e+00, 6.15848e+00, 2.06914e+01, 6.95193e+01, 2.33572e+02,
       7.84760e+02, 2.63665e+03, 8.85867e+03, 2.97635e+04, 1.00000e+05])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [8]:
print(gscv_reg_ridge.best_params_)
print(gscv_reg_ridge.best_score_)

{'alpha': 69.51927961775606}
0.8433555017333205


In [9]:
y_pred_rd = gscv_reg_ridge.predict(X_test_rd)

In [10]:
r2_rr = r2_score(y_test_rd, y_pred_rd)
rmse_rr = np.sqrt(mean_squared_error(y_test_rd, y_pred_rd))

In [11]:
print("R_squared_rr: {}".format(r2_rr))
print("RMSE_rr : {}".format(rmse_rr))

R_squared_rr: 0.8423917584053462
RMSE_rr : 9.781932233313192


### 3.3 Build API inputs

In [12]:
df_input_API = df_model_sc.drop(['CARRIER_DELAY', 'WEATHER_DELAY','NAS_DELAY','SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY','DEP_DELAY'],axis=1)
input_API = dict(zip(df_input_API.columns,range(df_input_API.shape[1])))

### 3.4 API inputs reset

In [13]:
input_API_0 = np.zeros(len(input_API))
#input_API_0

### 3.5 API inputs real test

In [14]:
input_API_0[input_API['MONTH_'+str(2)]] = 1
input_API_0[input_API['DAY_OF_MONTH_'+str(15)]] = 1
input_API_0[input_API['DAY_OF_WEEK_'+str(4)]] = 1
input_API_0[input_API['CARRIER_'+str('WN')]] = 1
input_API_0[input_API['ORIGIN_CITY_NAME_'+str('Atlanta, GA')]] = 1
input_API_0[input_API['DEST_CITY_NAME_'+str('Boston, MA')]] = 1
input_API_0[input_API['CRS_DEP_TIME_'+str(10)]] = 1

In [15]:
input_API_on = input_API_0
#input_API_on

In [16]:
input_API_on.shape

(627,)

### 3.6 Departure delay prediction

In [17]:

sc = StandardScaler()
delay_var_output = []
input_API_on_1 = input_API_on.reshape(-1, 1)

y_carrier_delay_pred_knn = gscv_reg_carrier_delay_knn.predict(input_API_on_1.T) #.append
delay_var_output.append(y_carrier_delay_pred_knn)


y_NAS_delay_pred_knn = gscv_reg_NAS_delay_knn.predict(input_API_on_1.T) #.append
delay_var_output.append(y_NAS_delay_pred_knn)


y_AIRCRAFT_delay_pred_knn = gscv_reg_AIRCRAFT_delay_knn.predict(input_API_on_1.T) #.append
delay_var_output.append(y_AIRCRAFT_delay_pred_knn)


delay_var_output = sc.fit_transform(delay_var_output)
    
def oneDArray(x):
    return list(itertools.chain(*x))
delay_var_output = oneDArray(delay_var_output)

input_Final = np.concatenate((input_API_on,delay_var_output))
input_Final = input_Final.reshape(-1,1)
dep_delay_pred = gscv_reg_ridge.predict(input_Final.T)
dep_delay_pred

array([2.84380609])

### 3.7 Save API information

In [18]:
# Python Module used to convert a python object into a character stream

In [20]:
#with open('gscv_reg_carrier_delay_knn.pkl', 'wb') as f:
    #pickle.dump(gscv_reg_carrier_delay_knn, f)
    
joblib.dump(gscv_reg_carrier_delay_knn, 'gscv_reg_carrier_delay_knn.joblib', compress=1)

#f=gzip.open('gscv_reg_carrier_delay_knn.pkl', 'wb') 
#pickle.dump(gscv_reg_carrier_delay_knn, f)

['gscv_reg_carrier_delay_knn.joblib']

In [21]:
#with open('gscv_reg_NAS_delay_knn.pkl', 'wb') as f:
    #pickle.dump(gscv_reg_NAS_delay_knn, f) 
    
joblib.dump(gscv_reg_NAS_delay_knn, 'gscv_reg_NAS_delay_knn.joblib', compress=1)

#f=gzip.open('gscv_reg_NAS_delay_knn.pkl', 'wb') 
#pickle.dump(gscv_reg_NAS_delay_knn, f)

['gscv_reg_NAS_delay_knn.joblib']

In [22]:
#with open('gscv_reg_AIRCRAFT_delay_knn.pkl', 'wb') as f:
    #cPickle.dump(gscv_reg_AIRCRAFT_delay_knn, f)
    
joblib.dump(gscv_reg_AIRCRAFT_delay_knn, 'gscv_reg_AIRCRAFT_delay_knn.joblib', compress=1)

#f=gzip.open('gscv_reg_AIRCRAFT_delay_knn.pkl', 'wb') 
#pickle.dump(gscv_reg_AIRCRAFT_delay_knn, f)

['gscv_reg_AIRCRAFT_delay_knn.joblib']

In [23]:
#with open('gscv_reg_ridge.pkl', 'wb') as f:
    #pickle.dump(gscv_reg_ridge, f)
    
joblib.dump(gscv_reg_ridge, 'gscv_reg_ridge.joblib', compress=1)

#f=gzip.open('gscv_reg_ridge.pkl', 'wb') 
#pickle.dump(gscv_reg_ridge, f)

['gscv_reg_ridge.joblib']

In [24]:
#with open('input_API.pkl', 'wb') as f:
    #pickle.dump(input_API, f)
    
joblib.dump(input_API, 'input_API.joblib', compress=1)

#f=gzip.open('input_API.pkl', 'wb') 
#pickle.dump(input_API, f)


['input_API.joblib']

In [25]:
#API_info = gzip.open('input_API.pkl', 'rb')
#input_API = pickle.load(API_info)
#input_API_0 = np.zeros(len(input_API))

API_info = open('input_API.joblib', 'rb')
input_API = joblib.load(API_info)
input_API_0 = np.zeros(len(input_API))

In [26]:
input_API_0

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.